In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3'
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from niwidgets import NiftiWidget
import tensorflow as tf
from tensorflow.keras.preprocessing.image import * 
from tensorflow import keras
from config import *
import math
import random
import numpy as np
from skimage.util import montage, crop
from medio import read_img, save_img
from tensorflow.python.client import device_lib
from tensorflow.keras.metrics import MeanIoU,BinaryCrossentropy,Accuracy,Precision,Recall
from skimage.transform import rescale, resize, downscale_local_mean,rotate
from skimage import data, color
from scipy import ndimage
from scipy import misc
from numpy import fliplr
from datetime import *

#print(device_lib.list_local_devices())
#tf.config.list_physical_devices('GPU')

In [16]:
def display(image,mask):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12, 6))
    ax1.imshow(test_image[test_image.shape[0]//2])
    ax1.set_title('Image')
    ax2.imshow(test_mask[test_image.shape[0]//2])
    ax2.set_title('Mask')
    
   
    fig, ax1 = plt.subplots(1, 1, figsize = (20, 20))
    ax1.imshow(montage(test_image), cmap ='bone')


    fig, ax1 = plt.subplots(1, 1, figsize = (20, 20))
    ax1.imshow(montage(test_mask), cmap ='bone')
    
    return None

In [17]:


# Datasets
train_dataset = data_dir_train_image
train_labels = data_dir_train_mask

i=0
#labels,meta =read_img(train_labels,backend='nib',channels_axis=int)
for niiImage in os.listdir(data_dir_train_image):
    #print(niiImage)
    inputs = nib.load( os.path.join(data_dir_train_image, niiImage))   
    mask = nib.load( os.path.join(data_dir_train_mask, niiImage)) 
    
    test_image=nib.load( os.path.join(data_dir_train_image, niiImage)).get_fdata()
    test_mask=nib.load( os.path.join(data_dir_train_mask, niiImage)).get_fdata()
    
   # display(test_image,test_mask)
  

In [18]:

def rescaleImage(aug_images,aug_masks,scale):
    
    rescaled_image = resize(rescale(aug_images[0], scale,anti_aliasing=False), (40, 40, 80), anti_aliasing=True) 
    rescaled_mask = resize(rescale(aug_masks[0], scale,anti_aliasing=False), (40, 40, 80), anti_aliasing=True)
   
    aug_images.append(rescaled_image)
    aug_masks.append(rescaled_mask)

    return (aug_images,aug_masks)

def rotateImage(aug_images,aug_masks,angle):
    
    rescaled_image = resize(rotate(aug_images[0], angle), (40, 40, 80), anti_aliasing=True) 
    rescaled_mask = resize(rotate(aug_masks[0], angle), (40, 40, 80), anti_aliasing=True)
   
    aug_images.append(rescaled_image)
    aug_masks.append(rescaled_mask)
    
    return (aug_images,aug_masks)

def dataAugmentation(aug_images,aug_masks,scale=None,rotationAngle=None):
    
    if(scale is not None):
        aug_images,aug_masks = rescaleImage(aug_images,aug_masks,scale)
    
    if(rotationAngle is not None):
        aug_images,aug_masks = rotateImage(aug_images,aug_masks,rotationAngle)
        aug_images,aug_masks = rotateImage(aug_images,aug_masks,-rotationAngle)
        
    return (aug_images,aug_masks)

def resize_data(image,shape=(40,40,80)):

    resize_image = resize(rescale(image, 1,anti_aliasing=False), shape, anti_aliasing=True) #inputs.get_fdata()[5:45, 5: 45, 10:90]   
    return (resize_image)
    
def train(SCALE,ROTATIONANGLE):
    aug_images = []
    aug_masks=[]
   
    for niiImage in os.listdir(data_dir_train_image):
      #  print(niiImage)
        inputs = nib.load( os.path.join(data_dir_train_image, niiImage)).get_fdata()   
        mask = nib.load( os.path.join(data_dir_train_mask, niiImage)).get_fdata() 

        aug_images.append(resize_data(inputs))
        aug_masks.append(resize_data(mask))
    
        tempList = dataAugmentation(aug_images,aug_masks,SCALE,ROTATIONANGLE)
         
        img_np = np.rollaxis(np.array(np.array([np.array(tempList[0])]) ),0,5)
        img_mask = np.rollaxis(np.array(np.array([np.array(tempList[1])]) ),0,5)
      
     #   print(len(img_np))
    return (img_np,img_mask)


In [19]:
#print(image_patches)

def unet(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, in_channels=1, out_channels=1):
    
    
    inputs =  keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH,IMAGE_DEPTH, in_channels))
    x = inputs
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv3D(initial_features * 2 ** level, **convpars)(x)
        x = keras.layers.BatchNormalization()(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool3D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv3DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv3D(initial_features * 2 ** level, **convpars)(x)
        x = keras.layers.BatchNormalization()(x)
      
    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv3D(out_channels, kernel_size=1, activation=activation, padding='same')(x)
    
    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET3D-L{n_levels}-F{initial_features}')

In [22]:
def main():
    model= unet(4)

    for r in ROTATIONANGLE:
        for s in {None,0.5,0.25,2.0}:
            path = 'model/UNET3D/'
            modelName = 'UNET3D-ToothSegmentation-scale_'+str(s)+'-rotation_'+str(r)+'.h5'
            model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
            #model.summary()

            i,m = train(s,r)
            batchSize= i.shape[0] 
            stepsPerEpoch =  1

            logdir = "logs/UNET3D/" + datetime.now().strftime("%Y%m%d-%H%M%S")+ modelName
            tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


            model.fit(i,m,batch_size=batchSize,epochs=500,steps_per_epoch=stepsPerEpoch,validation_split = 0.2,callbacks=[tensorboard_callback])
            model.save(path +modelName)
    return None

In [23]:
main()

The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.


Train on 22 samples, validate on 6 samples
Epoch 1/500
22/22 [==============================] - 4s 176ms/sample - loss: 0.2952 - accuracy: 0.4114 - val_loss: 0.8413 - val_accuracy: 0.0617
Epoch 2/500
22/22 [==============================] - 2s 82ms/sample - loss: 0.2356 - accuracy: 0.5915 - val_loss: 0.8519 - val_accuracy: 0.0681
Epoch 3/500
22/22 [==============================] - 1s 43ms/sample - loss: 0.1988 - accuracy: 0.6678 - val_loss: 0.8728 - val_accuracy: 0.0598
Epoch 4/500
22/22 [==============================] - 1s 43ms/sample - loss: 0.1733 - accuracy: 0.7192 - val_loss: 0.8921 - val_accuracy: 0.0488
Epoch 5/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.1528 - accuracy: 0.7424 - val_loss: 0.8960 - val_accuracy: 0.0475
Epoch 6/500
22/22 [==============================] - 1s 43ms/sample - loss: 0.1278 - accuracy: 0.7801 - val_loss: 0.8964 - val_accuracy: 0.0474
Epoch 7/500
22/22 [==============================] - 1s 43ms/sample - loss: 0.1108 - accurac

Epoch 57/500
22/22 [==============================] - 1s 46ms/sample - loss: 0.0141 - accuracy: 0.8575 - val_loss: 0.7366 - val_accuracy: 0.1975
Epoch 58/500
22/22 [==============================] - 1s 46ms/sample - loss: 0.0137 - accuracy: 0.8575 - val_loss: 0.7280 - val_accuracy: 0.2062
Epoch 59/500
22/22 [==============================] - 1s 47ms/sample - loss: 0.0133 - accuracy: 0.8575 - val_loss: 0.7171 - val_accuracy: 0.2165
Epoch 60/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.0130 - accuracy: 0.8575 - val_loss: 0.6895 - val_accuracy: 0.2431
Epoch 61/500
22/22 [==============================] - 1s 45ms/sample - loss: 0.0127 - accuracy: 0.8575 - val_loss: 0.6909 - val_accuracy: 0.2415
Epoch 62/500
22/22 [==============================] - 1s 45ms/sample - loss: 0.0124 - accuracy: 0.8575 - val_loss: 0.6672 - val_accuracy: 0.2648
Epoch 63/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.0121 - accuracy: 0.8575 - val_loss: 0.6490 - val_ac

Epoch 114/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.0050 - accuracy: 0.8575 - val_loss: 0.1348 - val_accuracy: 0.7736
Epoch 115/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.0049 - accuracy: 0.8575 - val_loss: 0.1359 - val_accuracy: 0.7729
Epoch 116/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.0049 - accuracy: 0.8575 - val_loss: 0.1241 - val_accuracy: 0.7827
Epoch 117/500
22/22 [==============================] - 1s 46ms/sample - loss: 0.0048 - accuracy: 0.8575 - val_loss: 0.1189 - val_accuracy: 0.7866
Epoch 118/500
22/22 [==============================] - 1s 47ms/sample - loss: 0.0047 - accuracy: 0.8575 - val_loss: 0.1223 - val_accuracy: 0.7836
Epoch 119/500
22/22 [==============================] - 1s 46ms/sample - loss: 0.0047 - accuracy: 0.8575 - val_loss: 0.1126 - val_accuracy: 0.7913
Epoch 120/500
22/22 [==============================] - 1s 47ms/sample - loss: 0.0046 - accuracy: 0.8575 - val_loss: 0.1027 -

Epoch 171/500
22/22 [==============================] - 1s 45ms/sample - loss: 0.0039 - accuracy: 0.8575 - val_loss: 0.0168 - val_accuracy: 0.8570
Epoch 172/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.0048 - accuracy: 0.8575 - val_loss: 0.0113 - val_accuracy: 0.8589
Epoch 173/500
22/22 [==============================] - 1s 45ms/sample - loss: 0.0032 - accuracy: 0.8575 - val_loss: 0.0080 - val_accuracy: 0.8595
Epoch 174/500
22/22 [==============================] - 1s 44ms/sample - loss: 0.0039 - accuracy: 0.8575 - val_loss: 0.0202 - val_accuracy: 0.8555
Epoch 175/500
22/22 [==============================] - 1s 46ms/sample - loss: 0.0035 - accuracy: 0.8575 - val_loss: 0.0184 - val_accuracy: 0.8562
Epoch 176/500
22/22 [==============================] - 1s 47ms/sample - loss: 0.0035 - accuracy: 0.8575 - val_loss: 0.0068 - val_accuracy: 0.8596
Epoch 177/500
22/22 [==============================] - 1s 47ms/sample - loss: 0.0033 - accuracy: 0.8575 - val_loss: 0.0074 -

...

Epoch 224/500
5/5 [==============================] - 0s 51ms/sample - loss: 3.3458e-05 - accuracy: 0.8977 - val_loss: 0.0091 - val_accuracy: 0.8947
Epoch 225/500
5/5 [==============================] - 0s 52ms/sample - loss: 3.4661e-05 - accuracy: 0.8977 - val_loss: 0.0092 - val_accuracy: 0.8947
Epoch 226/500
5/5 [==============================] - 0s 51ms/sample - loss: 2.9569e-05 - accuracy: 0.8977 - val_loss: 0.0093 - val_accuracy: 0.8946
Epoch 227/500
5/5 [==============================] - 0s 52ms/sample - loss: 3.1430e-05 - accuracy: 0.8977 - val_loss: 0.0093 - val_accuracy: 0.8946
Epoch 228/500
5/5 [==============================] - 0s 52ms/sample - loss: 3.3034e-05 - accuracy: 0.8977 - val_loss: 0.0094 - val_accuracy: 0.8946
Epoch 229/500
5/5 [==============================] - 0s 59ms/sample - loss: 2.9280e-05 - accuracy: 0.8977 - val_loss: 0.0095 - val_accuracy: 0.8946
Epoch 230/500
5/5 [==============================] - 0s 51ms/sample - loss: 2.9905e-05 - accuracy: 0.8977 - val_

Epoch 280/500
5/5 [==============================] - 0s 55ms/sample - loss: 3.0703e-05 - accuracy: 0.8977 - val_loss: 0.0133 - val_accuracy: 0.8933
Epoch 281/500
5/5 [==============================] - 0s 54ms/sample - loss: 3.4234e-05 - accuracy: 0.8977 - val_loss: 0.0137 - val_accuracy: 0.8931
Epoch 282/500
5/5 [==============================] - 0s 55ms/sample - loss: 2.5390e-05 - accuracy: 0.8977 - val_loss: 0.0139 - val_accuracy: 0.8931
Epoch 283/500
5/5 [==============================] - 0s 51ms/sample - loss: 2.8327e-05 - accuracy: 0.8977 - val_loss: 0.0135 - val_accuracy: 0.8932
Epoch 284/500
5/5 [==============================] - 0s 53ms/sample - loss: 3.1381e-05 - accuracy: 0.8977 - val_loss: 0.0140 - val_accuracy: 0.8930
Epoch 285/500
5/5 [==============================] - 0s 56ms/sample - loss: 2.4924e-05 - accuracy: 0.8977 - val_loss: 0.0142 - val_accuracy: 0.8929
Epoch 286/500
5/5 [==============================] - 0s 54ms/sample - loss: 2.6659e-05 - accuracy: 0.8977 - val_

Epoch 336/500
5/5 [==============================] - 0s 52ms/sample - loss: 1.9554e-05 - accuracy: 0.8977 - val_loss: 0.0215 - val_accuracy: 0.8885
Epoch 337/500
5/5 [==============================] - 0s 53ms/sample - loss: 1.9506e-05 - accuracy: 0.8977 - val_loss: 0.0217 - val_accuracy: 0.8883
Epoch 338/500
5/5 [==============================] - 0s 52ms/sample - loss: 1.9442e-05 - accuracy: 0.8977 - val_loss: 0.0219 - val_accuracy: 0.8882
Epoch 339/500
5/5 [==============================] - 0s 52ms/sample - loss: 1.9376e-05 - accuracy: 0.8977 - val_loss: 0.0221 - val_accuracy: 0.8881
Epoch 340/500
5/5 [==============================] - 0s 52ms/sample - loss: 1.9345e-05 - accuracy: 0.8977 - val_loss: 0.0223 - val_accuracy: 0.8879
Epoch 341/500
5/5 [==============================] - 0s 55ms/sample - loss: 1.9392e-05 - accuracy: 0.8977 - val_loss: 0.0225 - val_accuracy: 0.8877
Epoch 342/500
5/5 [==============================] - 0s 55ms/sample - loss: 1.9583e-05 - accuracy: 0.8977 - val_

Epoch 392/500
5/5 [==============================] - 0s 51ms/sample - loss: 1.7550e-05 - accuracy: 0.8977 - val_loss: 0.0346 - val_accuracy: 0.8788
Epoch 393/500
5/5 [==============================] - 0s 54ms/sample - loss: 1.8076e-05 - accuracy: 0.8977 - val_loss: 0.0348 - val_accuracy: 0.8787
Epoch 394/500
5/5 [==============================] - 0s 53ms/sample - loss: 1.7263e-05 - accuracy: 0.8977 - val_loss: 0.0349 - val_accuracy: 0.8785
Epoch 395/500
5/5 [==============================] - 0s 52ms/sample - loss: 1.7234e-05 - accuracy: 0.8977 - val_loss: 0.0352 - val_accuracy: 0.8783
Epoch 396/500
5/5 [==============================] - 0s 54ms/sample - loss: 1.7652e-05 - accuracy: 0.8977 - val_loss: 0.0353 - val_accuracy: 0.8781
Epoch 397/500
5/5 [==============================] - 0s 54ms/sample - loss: 1.7001e-05 - accuracy: 0.8977 - val_loss: 0.0356 - val_accuracy: 0.8778
Epoch 398/500
5/5 [==============================] - 0s 51ms/sample - loss: 1.6988e-05 - accuracy: 0.8977 - val_

Epoch 448/500
5/5 [==============================] - 0s 53ms/sample - loss: 1.9003e-05 - accuracy: 0.8977 - val_loss: 0.0473 - val_accuracy: 0.8676
Epoch 449/500
5/5 [==============================] - 0s 51ms/sample - loss: 1.5665e-05 - accuracy: 0.8977 - val_loss: 0.0474 - val_accuracy: 0.8674
Epoch 450/500
5/5 [==============================] - 0s 52ms/sample - loss: 1.6120e-05 - accuracy: 0.8977 - val_loss: 0.0470 - val_accuracy: 0.8678
Epoch 451/500
5/5 [==============================] - 0s 53ms/sample - loss: 1.7921e-05 - accuracy: 0.8977 - val_loss: 0.0478 - val_accuracy: 0.8670
Epoch 452/500
5/5 [==============================] - 0s 53ms/sample - loss: 1.5935e-05 - accuracy: 0.8977 - val_loss: 0.0479 - val_accuracy: 0.8669
Epoch 453/500
5/5 [==============================] - 0s 51ms/sample - loss: 1.5065e-05 - accuracy: 0.8977 - val_loss: 0.0477 - val_accuracy: 0.8671
Epoch 454/500
5/5 [==============================] - 0s 51ms/sample - loss: 1.6629e-05 - accuracy: 0.8977 - val_

###### 